In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover, IDF, IndexToString
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StringIndexerModel
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression, DecisionTreeClassifier, GBTClassifier
from pyspark.ml.classification import NaiveBayesModel, RandomForestClassificationModel, LogisticRegressionModel, DecisionTreeClassificationModel, GBTClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import string
from pyspark.sql.functions import concat,concat_ws

In [2]:
!spark-submit --driver-memory 3g --executor-memory 3g /data/py_file/run_test.py --input_file /data/test/test_file.txt

2022-07-10 18:11:23,955 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension console dejavu execute kernel
kernelspec migrate nbconvert nbextension notebook qtconsole ru

2022-07-10 18:12:00,449 ERROR util.Utils: Uncaught exception in thread Thread-4
java.io.IOException: Filesystem closed
	at org.apache.hadoop.hdfs.DFSClient.checkOpen(DFSClient.java:474)
	at org.apache.hadoop.hdfs.DFSClient.getFileInfo(DFSClient.java:1663)
	at org.apache.hadoop.hdfs.DistributedFileSystem$29.doCall(DistributedFileSystem.java:1582)
	at org.apache.hadoop.hdfs.DistributedFileSystem$29.doCall(DistributedFileSystem.java:1579)
	at org.apache.hadoop.fs.FileSystemLinkResolver.resolve(FileSystemLinkResolver.java:81)
	at org.apache.hadoop.hdfs.DistributedFileSystem.getFileStatus(DistributedFileSystem.java:1594)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1683)
	at org.apache.spark.scheduler.EventLoggingListener.stop(EventLoggingListener.scala:248)
	at org.apache.spark.SparkContext$$anonfun$stop$8$$anonfun$apply$mcV$sp$6.apply(SparkContext.scala:1960)
	at org.apache.spark.SparkContext$$anonfun$stop$8$$anonfun$apply$mcV$sp$6.apply(SparkContext.scala:1960)
	at scala.Op

In [4]:
train_df = spark.read.format("csv").option("inferSchema", True).option("header", True).option("delimiter","\t").load("/data/train/train_raw")
indexer = StringIndexer(inputCol="Category", outputCol="categoryIndex")
indexed = indexer.fit(train_df)
indexed.save(model_path + "string_indexer")

In [7]:
indexer = StringIndexerModel.load(model_path + "string_indexer")
indexed = indexer.transform(train_df)
indexed.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+-------------+
|  _c0|                 URL|               Title|             Summary|            Contents|                Date|   Author(s)|    Category|                Tags|categoryIndex|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+-------------+
|    0|https://laodong.v...|"Thông tin “Ngọc ...|Lâm Đồng - Lãnh đ...|Những ngày vừa qu...|Thứ sáu, 20/05/20...|Phương Nhiên|Bất động sản|['Lâm Đồng', 'Ngọ...|         12.0|
|    1|https://laodong.v...|Lỗ hổng trong việ...|TPHCM - Việc khôn...|"Theo thông tin t...|Thứ sáu, 20/05/20...|    Gia Miêu|Bất động sản|['Thủ Thiêm', 'Đấ...|         12.0|
|   10|https://laodong.v...|Giá trị các khu đ...|Đồng Nai - Trong ...|Theo đó, ở TP.Biê...|Thứ tư, 18/05/202...|HÀ ANH CHIẾN|Bất đ

In [9]:
len(indexer.labels)

23

In [15]:
test_df = spark.read.format("csv").option("inferSchema", True).option("header", True).option("delimiter","\t").load("/data/test/test_raw")
fullContents = test_df.select(concat_ws(' ', test_df.Title, test_df.Summary, test_df.Contents).alias("fullContents"), "Category")
only_str = fullContents.withColumn("only_str",regexp_replace(col('fullContents'), '\d+|“|”', ''))
only_letter = only_str.withColumn("only_letter",regexp_replace(col('only_str'), '\\p{Punct}', '')) 
regex_tokenizer = RegexTokenizer(inputCol="only_letter", outputCol="words")
raw_words = regex_tokenizer.transform(only_letter)
loadedCv = CountVectorizerModel.load(model_path + "/count_vectorizer")
countVectorizer_train = loadedCv.transform(raw_words)
idfModel = IDFModel.load(model_path + "/idf")
tf_idf_test = idfModel.transform(countVectorizer_train)
tf_idf_test.show(5)

[Stage 35:>                                                         (0 + 1) / 1]

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        fullContents|Category|            only_str|         only_letter|               words|            features|     tf_idf_features|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Hỗ trợ người lao ...| Thời sự|Hỗ trợ người lao ...|Hỗ trợ người lao ...|[hỗ, trợ, người, ...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|
|Đưa Hải Phòng trở...| Thời sự|Đưa Hải Phòng trở...|Đưa Hải Phòng trở...|[đưa, hải, phòng,...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|
|Lần đầu tiên Bến ...| Thời sự|Lần đầu tiên Bến ...|Lần đầu tiên Bến ...|[lần, đầu, tiên, ...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|
|Ninh Thuận có tân...| Thời sự|Ninh Thuận có tân...|Ninh Thuận có tân...|[ninh, thuận, có,...|(262144,[0,2,3,5,...|(262144,[0,2,3,5,...|
|Báo cáo Chính phủ...| Thời sự|Báo cáo Ch

In [16]:
nb_save = model_path + "nb_model"
nbModel = NaiveBayesModel.load(nb_save)
nb_predictions = nbModel.transform(tf_idf_test)
nb_predictions.show(5)

[Stage 39:>                                                         (0 + 1) / 1]

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|        fullContents|Category|            only_str|         only_letter|               words|            features|     tf_idf_features|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Hỗ trợ người lao ...| Thời sự|Hỗ trợ người lao ...|Hỗ trợ người lao ...|[hỗ, trợ, người, ...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|[-10939.582751288...|[0.0,0.0,0.0,0.0,...|       8.0|
|Đưa Hải Phòng trở...| Thời sự|Đưa Hải Phòng trở...|Đưa Hải Phòng trở...|[đưa, hải, phòng,...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|[-7780.4990075850...|[0.0,0.0,0.0,0.0,...|       8.0|
|Lần đầu tiên Bến ...| Thời sự|Lần đầu tiên Bến ..

In [17]:
indexer = StringIndexerModel.load(model_path + "string_indexer")
indexed = indexer.transform(nb_predictions)
indexed.show(5)

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------+
|        fullContents|Category|            only_str|         only_letter|               words|            features|     tf_idf_features|       rawPrediction|         probability|prediction|categoryIndex|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------+
|Hỗ trợ người lao ...| Thời sự|Hỗ trợ người lao ...|Hỗ trợ người lao ...|[hỗ, trợ, người, ...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|[-10939.582751288...|[0.0,0.0,0.0,0.0,...|       8.0|          8.0|
|Đưa Hải Phòng trở...| Thời sự|Đưa Hải Phòng trở...|Đưa Hải Phòng trở...|[đưa, hải, phòng,...|(262144,[0,1,2,3,...|(262144,[0,1,2,3,...|[-7780.4990075850...|[0.0,0.0,0.0,0.0,...|      

In [18]:
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(indexed)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))

[Stage 46:=============================>                            (1 + 1) / 2]

Accuracy of NaiveBayes is = 0.719695


In [3]:
only_str = fullContents.withColumn("only_str",regexp_replace(col('fullContents'), '\d+|“|”', ''))
only_str.show(5)

+--------------------+------------+-------------+--------------------+
|        fullContents|    Category|categoryIndex|            only_str|
+--------------------+------------+-------------+--------------------+
|"Thông tin “Ngọc ...|Bất động sản|         12.0|"Thông tin Ngọc T...|
|Lỗ hổng trong việ...|Bất động sản|         12.0|Lỗ hổng trong việ...|
|Giá trị các khu đ...|Bất động sản|         12.0|Giá trị các khu đ...|
|Những điểm nghẽn ...|Bất động sản|         12.0|Những điểm nghẽn ...|
|Cơ hội đặc biệt đ...|Bất động sản|         12.0|Cơ hội đặc biệt đ...|
+--------------------+------------+-------------+--------------------+
only showing top 5 rows



In [4]:
only_letter = only_str.withColumn("only_letter",regexp_replace(col('only_str'), '\\p{Punct}', '')) 
only_letter.show(5)

+--------------------+------------+-------------+--------------------+--------------------+
|        fullContents|    Category|categoryIndex|            only_str|         only_letter|
+--------------------+------------+-------------+--------------------+--------------------+
|"Thông tin “Ngọc ...|Bất động sản|         12.0|"Thông tin Ngọc T...|Thông tin Ngọc Tr...|
|Lỗ hổng trong việ...|Bất động sản|         12.0|Lỗ hổng trong việ...|Lỗ hổng trong việ...|
|Giá trị các khu đ...|Bất động sản|         12.0|Giá trị các khu đ...|Giá trị các khu đ...|
|Những điểm nghẽn ...|Bất động sản|         12.0|Những điểm nghẽn ...|Những điểm nghẽn ...|
|Cơ hội đặc biệt đ...|Bất động sản|         12.0|Cơ hội đặc biệt đ...|Cơ hội đặc biệt đ...|
+--------------------+------------+-------------+--------------------+--------------------+
only showing top 5 rows



In [5]:
regex_tokenizer = RegexTokenizer(inputCol="only_letter", outputCol="words")
raw_words = regex_tokenizer.transform(only_letter)
raw_words.show(5)

+--------------------+------------+-------------+--------------------+--------------------+--------------------+
|        fullContents|    Category|categoryIndex|            only_str|         only_letter|               words|
+--------------------+------------+-------------+--------------------+--------------------+--------------------+
|"Thông tin “Ngọc ...|Bất động sản|         12.0|"Thông tin Ngọc T...|Thông tin Ngọc Tr...|[thông, tin, ngọc...|
|Lỗ hổng trong việ...|Bất động sản|         12.0|Lỗ hổng trong việ...|Lỗ hổng trong việ...|[lỗ, hổng, trong,...|
|Giá trị các khu đ...|Bất động sản|         12.0|Giá trị các khu đ...|Giá trị các khu đ...|[giá, trị, các, k...|
|Những điểm nghẽn ...|Bất động sản|         12.0|Những điểm nghẽn ...|Những điểm nghẽn ...|[những, điểm, ngh...|
|Cơ hội đặc biệt đ...|Bất động sản|         12.0|Cơ hội đặc biệt đ...|Cơ hội đặc biệt đ...|[cơ, hội, đặc, bi...|
+--------------------+------------+-------------+--------------------+--------------------+-----

In [7]:
cv = CountVectorizer(inputCol="words", outputCol="features")
model = cv.fit(raw_words)
model.save(model_path + "/count_vectorizer")

2022-07-10 15:13:02,384 WARN scheduler.TaskSetManager: Stage 11 contains a task of very large size (5508 KB). The maximum recommended task size is 100 KB.


In [10]:
loadedCv = CountVectorizerModel.load(model_path + "/count_vectorizer")
countVectorizer_train = model.transform(raw_words)
countVectorizer_train.show(5)

[Stage 17:>                                                         (0 + 1) / 1]

+--------------------+------------+-------------+--------------------+--------------------+--------------------+--------------------+
|        fullContents|    Category|categoryIndex|            only_str|         only_letter|               words|            features|
+--------------------+------------+-------------+--------------------+--------------------+--------------------+--------------------+
|"Thông tin “Ngọc ...|Bất động sản|         12.0|"Thông tin Ngọc T...|Thông tin Ngọc Tr...|[thông, tin, ngọc...|(262144,[0,1,2,3,...|
|Lỗ hổng trong việ...|Bất động sản|         12.0|Lỗ hổng trong việ...|Lỗ hổng trong việ...|[lỗ, hổng, trong,...|(262144,[0,1,2,3,...|
|Giá trị các khu đ...|Bất động sản|         12.0|Giá trị các khu đ...|Giá trị các khu đ...|[giá, trị, các, k...|(262144,[0,2,3,4,...|
|Những điểm nghẽn ...|Bất động sản|         12.0|Những điểm nghẽn ...|Những điểm nghẽn ...|[những, điểm, ngh...|(262144,[0,1,2,3,...|
|Cơ hội đặc biệt đ...|Bất động sản|         12.0|Cơ hội đặc bi

In [11]:
idf = IDF(inputCol="features", outputCol="tf_idf_features")
idfModel = idf.fit(countVectorizer_train)
idfModel.save(model_path + "/idf")

2022-07-10 15:21:15,437 WARN scheduler.TaskSetManager: Stage 21 contains a task of very large size (2098 KB). The maximum recommended task size is 100 KB.


In [3]:
text = spark.read.text("/data/test/test_file.txt")
text.show()

[Stage 0:>                                                          (0 + 1) / 1]

+--------------------+
|               value|
+--------------------+
|Đơn cử như mới đâ...|
+--------------------+



In [5]:
import sys
import argparse

In [3]:
model_path = "/data/model/"
text_df = spark.read.text("/data/test/test_file.txt")
indexer = StringIndexerModel.load(model_path + "string_indexer")
inverter = IndexToString(inputCol="prediction", outputCol="originalLabel", labels=indexer.labels)
indexed = indexer.transform(text_df)

only_str = indexed.withColumn("only_str",regexp_replace(col('value'), '\d+|“|”', ''))
only_letter = only_str.withColumn("only_letter",regexp_replace(col('only_str'), '\\p{Punct}', '')) 
regex_tokenizer = RegexTokenizer(inputCol="only_letter", outputCol="words")
raw_words = regex_tokenizer.transform(only_letter)

loadedCv = CountVectorizerModel.load(model_path + "/count_vectorizer")
countVectorizer_train = loadedCv.transform(raw_words)
idfModel = IDFModel.load(model_path + "/idf")
tf_idf_test = idfModel.transform(countVectorizer_train)

nb_save = model_path + "nb_model"
nbModel = NaiveBayesModel.load(nb_save)
nb_predictions = nbModel.transform(tf_idf_test)
nb_predictions = inverter.transform(nb_predictions)
category = nb_predictions.select("originalLabel").collect()[0][0]
print(category)

[Stage 25:>                                                         (0 + 1) / 1]

Bất động sản


In [1]:
import pandas as pd

In [16]:
df = pd.read_csv("test/test_raw/part-00000-b5754aec-c329-486e-b016-dd2c6d7b72c4-c000.csv", delimiter="\t", encoding="utf-8")

In [22]:
f = open("test/test_f.txt", "w", encoding="utf-16")
f.write(list(df.Contents)[12])
f.close()
list(df.Contents)[12]

'Đơn cử như mới đây, Sở Xây dựng TP.HCM vừa có văn bản tuýt còi về việc bán nhà ở hình thành trong tương lai đối với 204 căn hộ thuộc dự án Chung cư cao tầng Thảo Điền (tên thương mại là Thảo Điền Green), toạ lạc tại số 192 Nguyễn Văn Hưởng, phường Thảo Điền, TP.Thủ Đức. Dự án do Công ty cổ phần đầu tư Bất động sản SIC (Công ty SIC) làm chủ đầu tư từ ngày 8.3, thay cho Công ty cổ phần đầu tư Văn Phú - Invest.  Sở Xây dựng TP.HCM cho biết, do chủ đầu tư đã thế chấp nhà ở hình thành trong tương lai nên trước khi bán hay cho thuê, đơn vị này phải thay đổi nội dung thế chấp để rút bớt tài sản (trừ trường hợp chủ đầu tư, bên mua và bên nhận thế chấp có thỏa thuận khác). Chủ đầu tư phải đảm bảo điều kiện theo yêu cầu để được ngân hàng bảo lãnh nghĩa vụ tài chính của nhà đầu tư đối với khách hàng trong trường hợp không bàn giao nhà ở theo đúng tiến độ cam kết. Sở Xây dựng TPHCM đã yêu cầu chủ đầu tư huy động vốn lần đầu không quá 30% giá trị hợp đồng. Khi chưa bàn giao nhà cho khách hàng, nhữ

In [11]:
df_bds = df.where(df.Category == "Bất động sản")
df_bds.head(5)

,_c0,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Decision Tree

In [11]:
dt = DecisionTreeClassifier(featuresCol = 'tf_idf_features', labelCol = 'categoryIndex', maxDepth = 3)
dtModel = dt.fit(train)

In [13]:
dt_save = model_path + "dt_model"
dtModel.save(dt_save)

In [14]:
dtModel = DecisionTreeClassificationModel.load(dt_save)
dt_predictions = dtModel.transform(validate)
dt_predictions.show(5)

+--------------------+-------------+--------------------+--------------------+----------+
|     tf_idf_features|categoryIndex|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|(262144,[0,1,2,3,...|          6.0|[2374.0,26.0,790....|[0.14384391662627...|       6.0|
|(262144,[0,1,2,3,...|          0.0|[32353.0,3672.0,1...|[0.23033931851514...|       0.0|
|(262144,[0,1,2,3,...|          0.0|[2374.0,26.0,790....|[0.14384391662627...|       6.0|
|(262144,[0,1,2,3,...|          0.0|[2374.0,26.0,790....|[0.14384391662627...|       6.0|
|(262144,[0,1,2,3,...|          5.0|[32353.0,3672.0,1...|[0.23033931851514...|       0.0|
+--------------------+-------------+--------------------+--------------------+----------+
only showing top 5 rows



In [15]:
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dt_predictions)
print("Accuracy of Decision Tree is = %g"% (dt_accuracy))

[Stage 38:=============================>                            (1 + 1) / 2]

Accuracy of Decision Tree is = 0.344771


In [16]:
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="f1")
dt_f1 = evaluator.evaluate(dt_predictions)
print("F1 of Decision Tree is = %g"% (dt_f1))

[Stage 44:=============================>                            (1 + 1) / 2]

F1 of Decision Tree is = 0.252727


### Logistic Regression

In [17]:
lr_save = model_path + "lr_model"
lr = LogisticRegression(labelCol="categoryIndex", featuresCol="tf_idf_features", maxIter=10)
lrModel = lr.fit(train)
lrModel.save(lr_save)

2022-07-10 14:29:01,720 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2022-07-10 14:29:01,721 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
[Stage 50:=============================>                            (1 + 1) / 2]2022-07-10 14:29:14,043 ERROR util.Utils: Uncaught exception in thread task-result-getter-3
java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newArray(Native Method)
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2057)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1655)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2405)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2329)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2187)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)


KeyboardInterrupt: 

In [18]:
fullContents = spark.read.format("csv").option("inferSchema", True).option("header", True).option("delimiter","\t").load("/data/train/fullContents")
fullContents.show(5)

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:32869)

,Unnamed: 0,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
313315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df

,Unnamed: 0,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...,...,...
313315,313315,https://laodong.vn/tlv-canh-doi/ba-lao-mu-nuoi...,"Bà lão mù nuôi con, cháu bị ảnh hưởng chất độc...","Bà Dương Thị Tuyết ở thị trấn Khe Sanh, huyện ...",NaN,"Thứ tư, 14/08/2013 06:57 (GMT+7)",Lâm Hưng Thơ,Tấm Lòng Vàng,['Chất độc']
313316,313316,https://laodong.vn/ho-tro-bao-ve-chu-quyen-bie...,Trao 100 triệu đồng cho ngư dân Phú Yên,"Ngày 25.7, Đại diện Chương trình “Tấm lưới ngh...",NaN,"Thứ sáu, 26/07/2013 09:03 (GMT+7)",Lưu Phong,Tin hoạt động,['Phú Yên']
313317,313317,https://laodong.vn/giup-do-cac-canh-doi-kho-kh...,Trao 2 “Mái ấm công đoàn” tới CNVCLĐ khó khăn ...,"Ngày 17.7, lãnh đạo Quỹ TLV Lao Động và LĐLĐ t...",NaN,"Thứ năm, 18/07/2013 07:37 (GMT+7)",Bảo Duy,Tin hoạt động,"['Mái ấm Công đoàn', 'Bắc Giang']"
313318,313318,https://laodong.vn/tlv-canh-doi/hay-giup-do-mo...,Hãy giúp đỡ một nhà báo cần thay thận,Nhà báo Trần Đình Chính (tức Trần Hoài Thu) - ...,NaN,"Thứ năm, 20/06/2013 07:28 (GMT+7)",Hoàng Mạnh - Phạm Tùng,Tấm Lòng Vàng,"['Nhà báo', 'Báo']"
